# 02-视觉理解测试

## GLM-4.6V-Flash 模型能力

根据官方文档，GLM-4.6V-Flash 具备以下能力：

| 能力 | 支持情况 | 说明 |
|:---|:---:|:---|
| **深度思考** | ✅ 支持 | 支持开启或关闭思考模式 |
| **视觉理解** | ✅ 支持 | 支持图片、视频、文件 |
| **流式输出** | ✅ 支持 | 实时流式响应 |
| **Function Call** | ✅ 支持 | 原生多模态工具调用 |
| **上下文缓存** | ✅ 支持 | 优化长对话性能 |

## 模型规格

| 规格项 | 数值 |
|:---|:---|
| 输入模态 | 视频、图像、文本、文件 |
| 输出模态 | 文本 |
| 上下文窗口 | 128K |

## ⚠️ 重要限制

> **不支持同时理解文件、视频和图像** - 每次调用只能选择一种多模态类型

## 测试内容
1. 环境配置
2. 图片理解（单张/多张/Base64）
3. 视频理解
4. 文件理解
5. 思考模式测试
6. 流式输出测试
7. 多模态工具调用（Function Call）
8. 错误处理

## 1. 环境配置

In [ ]:
# 安装依赖
# !pip install openai python-dotenv requests

In [ ]:
import os
import base64
import requests
from openai import OpenAI

# 加载环境变量
try:
    from dotenv import load_dotenv
    load_dotenv()
    api_key = os.getenv('VITE_ZHIPU_API_KEY')
except:
    api_key = None

# 如果环境变量中没有，使用默认 API Key
if not api_key:
    api_key = "f7ec61a7856a4f4389e037eb588c69df.0pZgcFlTCk9GHCXM"

BASE_URL = "https://open.bigmodel.cn/api/paas/v4"

# 初始化客户端
client = OpenAI(
    api_key=api_key,
    base_url=BASE_URL
)

# 测试图片和视频 URL
TEST_IMAGE_URL = "https://cdn.bigmodel.cn/static/logo/register.png"
TEST_IMAGE_URL2 = "https://cdn.bigmodel.cn/static/logo/api-key.png"
TEST_VIDEO_URL = "https://cdn.bigmodel.cn/agent-demos/lark/113123.mov"
TEST_FILE_URL = "https://cdn.bigmodel.cn/static/demo/demo1.pdf"

print("✅ 客户端初始化成功！")
print(f"测试图片：{TEST_IMAGE_URL}")
print(f"测试视频：{TEST_VIDEO_URL}")

## 2. 图片理解测试

In [ ]:
# 2.1 单张图片理解
print("📷 测试1：单张图片理解\n")

response = client.chat.completions.create(
    model="glm-4.6v-flash",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {"url": TEST_IMAGE_URL}
                },
                {
                    "type": "text",
                    "text": "请描述这张图片的内容"
                }
            ]
        }
    ]
)

print("🤖 模型回复：")
print(response.choices[0].message.content)
print(f"\n📊 Token使用：{response.usage.total_tokens}")

In [ ]:
# 2.2 多张图片对比
print("📷📷 测试2：多张图片对比分析\n")

response = client.chat.completions.create(
    model="glm-4.6v-flash",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {"url": TEST_IMAGE_URL}
                },
                {
                    "type": "image_url",
                    "image_url": {"url": TEST_IMAGE_URL2}
                },
                {
                    "type": "text",
                    "text": "这两张图片有什么关联？请分别描述并对比"
                }
            ]
        }
    ]
)

print("🤖 模型回复：")
print(response.choices[0].message.content)

In [ ]:
# 2.3 Base64 图片输入
print("📷 测试3：Base64 编码图片\n")

# 下载图片并转换为 Base64
try:
    response_img = requests.get(TEST_IMAGE_URL, timeout=30)
    image_base64 = base64.b64encode(response_img.content).decode('utf-8')
    
    response = client.chat.completions.create(
        model="glm-4.6v-flash",
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {"url": image_base64}
                    },
                    {
                        "type": "text",
                        "text": "请描述这个图片"
                    }
                ]
            }
        ]
    )
    
    print("✅ Base64 图片输入成功")
    print("🤖 模型回复：")
    print(response.choices[0].message.content)
    
except Exception as e:
    print(f"❌ Base64 测试失败：{e}")

In [ ]:
# 2.4 图片 OCR（文字识别）
print("📷 测试4：图片 OCR 文字识别\n")

# 使用一张包含文字的图片
OCR_IMAGE_URL = "https://cdn.bigmodel.cn/static/logo/register.png"

response = client.chat.completions.create(
    model="glm-4.6v-flash",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {"url": OCR_IMAGE_URL}
                },
                {
                    "type": "text",
                    "text": "请提取图片中的所有文字内容"
                }
            ]
        }
    ]
)

print("🤖 OCR 结果：")
print(response.choices[0].message.content)

## 3. 视频理解测试

In [ ]:
# 3.1 视频内容描述
print("🎬 测试：视频内容理解\n")

response = client.chat.completions.create(
    model="glm-4.6v-flash",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "video_url",
                    "video_url": {"url": TEST_VIDEO_URL}
                },
                {
                    "type": "text",
                    "text": "这个视频展示了什么内容？请详细描述"
                }
            ]
        }
    ],
    max_tokens=2048
)

print("🤖 视频分析：")
print(response.choices[0].message.content)

In [ ]:
# 3.2 视频问答
print("🎬 测试：视频问答\n")

response = client.chat.completions.create(
    model="glm-4.6v-flash",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "video_url",
                    "video_url": {"url": TEST_VIDEO_URL}
                },
                {
                    "type": "text",
                    "text": "视频中的主要动作是什么？发生在什么时间点？"
                }
            ]
        }
    ],
    max_tokens=2048
)

print("🤖 视频问答结果：")
print(response.choices[0].message.content)

## 4. 文件理解测试

In [ ]:
# 4.1 单文件理解
print("📄 测试：PDF 文件理解\n")

response = client.chat.completions.create(
    model="glm-4.6v-flash",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "file_url",
                    "file_url": {"url": TEST_FILE_URL}
                },
                {
                    "type": "text",
                    "text": "请总结这个文档的主要内容"
                }
            ]
        }
    ],
    max_tokens=2048
)

print("🤖 文档摘要：")
print(response.choices[0].message.content)

## 5. 思考模式测试

In [ ]:
# 5.1 图片理解 + 思考模式
print("🧠📷 测试：图片理解 + 思考模式\n")

response = client.chat.completions.create(
    model="glm-4.6v-flash",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {"url": TEST_IMAGE_URL}
                },
                {
                    "type": "text",
                    "text": "请详细分析这张图片的设计元素和可能的设计意图"
                }
            ]
        }
    ],
    thinking={"type": "enabled"},
    max_tokens=4096
)

message = response.choices[0].message

# 显示思维链
if hasattr(message, 'reasoning_content') and message.reasoning_content:
    print("🧠 思维链：")
    print("-" * 60)
    print(message.reasoning_content)
    print("-" * 60)
    print("\n")

print("💡 最终分析：")
print(message.content)

## 6. 流式输出测试

In [ ]:
# 6.1 图片理解的流式输出
print("🌊📷 测试：图片理解 + 流式输出\n")
print("🤖 模型回复：", end="", flush=True)

response = client.chat.completions.create(
    model="glm-4.6v-flash",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {"url": TEST_IMAGE_URL}
                },
                {
                    "type": "text",
                    "text": "描述这张图片"
                }
            ]
        }
    ],
    stream=True
)

for chunk in response:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="", flush=True)

print("\n\n✅ 流式输出完成")

## 7. 错误处理测试

In [ ]:
# 7.1 测试：同时传入图片和视频（应该失败）
print("⚠️ 测试：同时传入图片和视频（预期会失败）\n")

try:
    response = client.chat.completions.create(
        model="glm-4.6v-flash",
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {"url": TEST_IMAGE_URL}
                    },
                    {
                        "type": "video_url",
                        "video_url": {"url": TEST_VIDEO_URL}
                    },
                    {
                        "type": "text",
                        "text": "分析这个图片和视频"
                    }
                ]
            }
        ]
    )
    print("❌ 应该失败但没有")
except Exception as e:
    print(f"✅ 正确捕获错误：{type(e).__name__}")
    print(f"   错误：{str(e)[:100]}...")

In [ ]:
# 7.2 测试：无效图片 URL
print("\n⚠️ 测试：无效图片 URL\n")

try:
    response = client.chat.completions.create(
        model="glm-4.6v-flash",
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {"url": "https://invalid-url.com/image.png"}
                    },
                    {
                        "type": "text",
                        "text": "描述图片"
                    }
                ]
            }
        ]
    )
    print("❌ 应该失败但没有")
except Exception as e:
    print(f"✅ 正确捕获错误：{type(e).__name__}")

## 8. 总结

In [ ]:
print("""
╔════════════════════════════════════════════════════════════════╗
║                  GLM-4.6V-Flash 测试总结                       ║
╠════════════════════════════════════════════════════════════════╣
║ 模型规格：                                                      ║
║   • 输入模态：视频、图像、文本、文件                             ║
║   • 输出模态：文本                                              ║
║   • 上下文窗口：128K tokens                                     ║
╠════════════════════════════════════════════════════════════════╣
║ 核心能力：                                                      ║
║   ✅ 深度思考 - 支持开启/关闭思考模式                            ║
║   ✅ 视觉理解 - 图片、视频、文件多模态理解                        ║
║   ✅ 流式输出 - 实时响应                                        ║
║   ✅ Function Call - 原生多模态工具调用                          ║
║   ✅ 上下文缓存 - 长对话优化                                     ║
╠════════════════════════════════════════════════════════════════╣
║ 重要限制：                                                      ║
║   ⚠️ 不支持同时理解文件、视频和图像                              ║
║   ⚠️ 每次调用只能选择一种多模态类型                              ║
╠════════════════════════════════════════════════════════════════╣
║ 适用场景：                                                      ║
║   • 图片理解 - OCR、表格解析、商品识别、缺陷检测                  ║
║   • 视频理解 - 内容标签、关键帧提取、视频问答、事件时间轴          ║
║   • 文档理解 - 复杂版式、智能问答、多文档关联分析                  ║
║   • 多模态 Agent - 图文混排、识图购物、前端复刻                  ║
╚════════════════════════════════════════════════════════════════╝
""")